In [1]:
import os
import numpy as np
from rmgpy.rmg.main import RMG, RMG_Memory, log_conditions
import rmgpy.chemkin
from rmgpy.rmg.model import ReactionModel, CoreEdgeReactionModel
from rmgpy import settings
from rmgpy.data.rmg import RMGDatabase
from rmgpy.molecule import Molecule
from rmgpy.species import Species

In [2]:
#load the RMG database
database = RMGDatabase()

database.load(
    path = settings['database.directory'],
    thermo_libraries = ['primaryThermoLibrary'] ,  # Can add others if necessary
    kinetics_families = 'default',
    kinetics_depositories = ['training'],
)

In [3]:
#load the correct group additivity values, hydrogen bond increment values, single ring corrections
database.thermo.load_groups('/home/khalil.nor/Code/RMG-database/input/thermo/groups/')
#make sure everything is loaded
database.thermo.groups

{'group': <rmgpy.data.thermo.ThermoGroups at 0x2b831b74a7d0>,
 'ring': <rmgpy.data.thermo.ThermoGroups at 0x2b8221f1a090>,
 'radical': <rmgpy.data.thermo.ThermoGroups at 0x2b831b837ad0>,
 'polycyclic': <rmgpy.data.thermo.ThermoGroups at 0x2b831ceba3d0>,
 'other': <rmgpy.data.thermo.ThermoGroups at 0x2b831cee0dd0>,
 'longDistanceInteraction_cyclic': <rmgpy.data.thermo.ThermoGroups at 0x2b831cef6d90>,
 'longDistanceInteraction_noncyclic': <rmgpy.data.thermo.ThermoGroups at 0x2b831cef6dd0>,
 'adsorptionPt111': <rmgpy.data.thermo.ThermoGroups at 0x2b831cf04590>}

In [72]:
#load the molecules 
cyclohexyl_radical = Molecule(smiles='[CH]1CCCCC1')
hexen_6_yl_radical = Molecule(smiles='C=CCCC[CH2]')

In [7]:
#this function in RMG accounts for HBI, ring corrections, and symmetry number 
thermo_data_hexen_6_yl = database.thermo.estimate_thermo_via_group_additivity(hexen_6_yl_radical)
thermo_data_cyclohexyl = database.thermo.estimate_thermo_via_group_additivity(cyclohexyl_radical)


# Question 1 

**Using the RMG estimation methods, the thermochemical parameters for these two species are provided below:** 

**For cyclohexyl radical, the JetSurF1.0 library reports (from RMG website)**
* H(298) = 17.00 kcal/mol 
* S(298) = 77.80 cal/(mol*K)
* Cp(298) = 25.82 cal/(mol*K)

In [5]:
cyclohexyl_species = Species(index=0, label='cyc-rad', molecule=[Molecule(smiles='[CH]1CCCCC1')])
cyclohexyl_thermo_parameters = database.thermo.get_thermo_data_from_groups(cyclohexyl_species)
cyclohexyl_thermo_parameters.to_nasa(298, 5000, 1000)

NASA(polynomials=[NASAPolynomial(coeffs=[-5.18775,0.0677176,-2.83289e-05,-7.01326e-10,2.34008e-12,5782.56,50.6646], Tmin=(298,'K'), Tmax=(1755.73,'K')), NASAPolynomial(coeffs=[55.7201,-0.0223193,6.96412e-06,1.70478e-09,-2.53343e-13,-23115.2,-298.852], Tmin=(1755.73,'K'), Tmax=(5000,'K'))], Tmin=(298,'K'), Tmax=(5000,'K'), E0=(39.9688,'kJ/mol'), Cp0=(33.2579,'J/(mol*K)'), CpInf=(407.409,'J/(mol*K)'), comment="""Thermo group additivity estimation: group(Cs-CsCsHH) + group(Cs-CsCsHH) + group(Cs-CsCsHH) + group(Cs-CsCsHH) + group(Cs-CsCsHH) + group(Cs-CsCsHH) + ring(Cyclohexane) + radical(cyclohexane)""")

In [8]:
thermo_data_cyclohexyl

ThermoData(Tdata=([300,400,500,600,800,1000,1500],'K'), Cpdata=([106.65,144.181,178.573,211.166,262.713,298.236,349.699],'J/(mol*K)'), H298=(58.145,'kJ/mol'), S298=(332.828,'J/(mol*K)'), comment="""Thermo group additivity estimation: group(Cs-CsCsHH) + group(Cs-CsCsHH) + group(Cs-CsCsHH) + group(Cs-CsCsHH) + group(Cs-CsCsHH) + group(Cs-CsCsHH) + ring(Cyclohexane) + radical(cyclohexane)""")

**Two cells above, we calculate the thermo parameters for the cyclohexyl radical using group additivity and cast it into NASA polynomial form**


**From above, we can see that group additivity estimated for the cyclohexyl radical:**
* H(298) = 58.145 kJ/mol = 13.8969885 kcal/mol
* S(298) = 332.828 J/(mol*K) = 79.54780115 cal/(mol*K)
* Cp(300) = 106.65 J/(mol*K) = 25.4899618 cal/(mol*K)

**Therefore, the group additivity thermo values that I estimated are somewhat close to the reported thermos.**

In [12]:
#now let's calculate the thermo parameters using group additivity
cyclohexyl_species = Species(index=0, label='cyc-rad', molecule=[Molecule(smiles='[CH]1CCCCC1')])
cyclohexyl_thermo_parameters = database.thermo.get_thermo_data_from_groups(hexen_6_yl_species)
cyclohexyl_thermo_parameters.to_nasa(298, 5000, 1000)

NASA(polynomials=[NASAPolynomial(coeffs=[-0.382061,0.064079,-4.11571e-05,1.34115e-08,-1.75369e-12,17245.2,33.7267], Tmin=(298,'K'), Tmax=(1791.19,'K')), NASAPolynomial(coeffs=[16.6205,0.0261098,-9.36056e-06,1.57717e-09,-1.01944e-13,11154.2,-58.2113], Tmin=(1791.19,'K'), Tmax=(5000,'K'))], Tmin=(298,'K'), Tmax=(5000,'K'), E0=(140.978,'kJ/mol'), Cp0=(33.2579,'J/(mol*K)'), CpInf=(390.78,'J/(mol*K)'), comment="""Thermo group additivity estimation: group(Cs-CsCsHH) + group(Cs-CsCsHH) + group(Cs-(Cds-Cds)CsHH) + group(Cs-CsHHH) + group(Cds-CdsCsH) + group(Cds-CdsHH) + radical(RCCJ)""")

In [13]:
# Ugh, the RMG website does not have the interactive thermo plots for hexen_6_yl radical, so let's calculate it manually. Ugh x2

# parameters in temperature range of 298 to 1000K 
a_neg_2 = 0
a_neg_1 = 0
a0 = -8.10229e-1
a1 = 6.88802e-2
a2 = -4.93843e-5
a3 = 1.71881e-8
a4 = -1.64486e-12
a5 = 1.70719e4
a6 = 3.32599e1

# J / mol K
R = 8.31446261815324
T = 298 #K

Cp_JetSurf_hexen_6_yl = R * (a_neg_2*(T**-2)+a_neg_1*(T**-1)+a0+a1*T+a2*(T**2)+a3*(T**3)+a4*(T**4)) # J/ mol K
H_JetSurf_hexen_6_yl = R*T * (-a_neg_2*(T**-2)+a_neg_1*(np.log(T)/T)+a0+(1/2)*a1*T+(1/3)*a2*(T**2)+(1/4)*a3*(T**3)+(1/5)*a4*(T**4)+(a5/T)) #J / mol
S_JetSurf_hexen_6_yl = R * ((-1/2)*a_neg_2*(T**-2)-a_neg_1*(T**-1)+a0*np.log(T)+a1*T+(1/2)*a2*(T**2)+(1/3)*a3*(T**3)+(1/4)*a4*(T**4)+a6) #J/ mol K

print(f'At 298 K, H(298) = {H_JetSurf_hexen_6_yl/1000} kJ/ mol, S(298) = {S_JetSurf_hexen_6_yl} J/mol K, Cp(298) = {Cp_JetSurf_hexen_6_yl} J/mol K')

At 298 K, H(298) = 162.01857536873973 kJ/ mol, S(298) = 391.8262368332505 J/mol K, Cp(298) = 131.1393168403402 J/mol K


**For hexen_6_yl radical, the JetSurF1.0 library reports (calculated manually above)**

* H(298) = 162.01857536873973 kJ/ mol
* S(298) = 391.8262368332505 J/mol K 
* Cp(298) = 131.1393168403402 J/mol K


In [6]:
#now let's calculate the thermo parameters using group additivity
hexen_6_yl_species = Species(index=0, label='hex-rad', molecule=[Molecule(smiles='C=CCCC[CH2]')])
hexen_6_yl_thermo_parameters = database.thermo.get_thermo_data_from_groups(hexen_6_yl_species)
hexen_6_yl_thermo_parameters.to_nasa(298, 5000, 1000)

NASA(polynomials=[NASAPolynomial(coeffs=[-0.382061,0.064079,-4.11571e-05,1.34115e-08,-1.75369e-12,17245.2,33.7267], Tmin=(298,'K'), Tmax=(1791.19,'K')), NASAPolynomial(coeffs=[16.6205,0.0261098,-9.36056e-06,1.57717e-09,-1.01944e-13,11154.2,-58.2113], Tmin=(1791.19,'K'), Tmax=(5000,'K'))], Tmin=(298,'K'), Tmax=(5000,'K'), E0=(140.978,'kJ/mol'), Cp0=(33.2579,'J/(mol*K)'), CpInf=(390.78,'J/(mol*K)'), comment="""Thermo group additivity estimation: group(Cs-CsCsHH) + group(Cs-CsCsHH) + group(Cs-(Cds-Cds)CsHH) + group(Cs-CsHHH) + group(Cds-CdsCsH) + group(Cds-CdsHH) + radical(RCCJ)""")

In [9]:
thermo_data_hexen_6_yl

ThermoData(Tdata=([300,400,500,600,800,1000,1500],'K'), Cpdata=([128.867,161.628,190.916,215.811,254.973,284.135,328.821],'J/(mol*K)'), H298=(163.289,'kJ/mol'), S298=(406.852,'J/(mol*K)'), comment="""Thermo group additivity estimation: group(Cs-CsCsHH) + group(Cs-CsCsHH) + group(Cs-(Cds-Cds)CsHH) + group(Cs-CsHHH) + group(Cds-CdsCsH) + group(Cds-CdsHH) + radical(RCCJ)""")

**Two cells above, we calculate the thermo parameters for the hexen_6_yl radical using group additivity and cast it into NASA polynomial form**

**From one cell above, we can see that group additivity estimated for the hexen_6_yl radical:**

* H(298) = 163.289 kJ/mol = 39.02700765 kcal/mol
* S(298) = 406.852 J/(molK) = 97.23996176 cal/(molK)
* Cp(300) = 128.867 J/(molK) = 30.7999522 cal/(molK)

**Therefore, the group additivity thermo values that I estimated are somewhat close to the reported thermos.**

Now let's simulate in Cantera

In [31]:
import cantera as ct
print(f'Running Cantera {ct.__version__}')

Running Cantera 2.6.0


In [54]:
#compute the equilibrium constant
#Keq = exp(-delta_g_std_rxn/RT) = 1 if delta_g_std_rxn is 0


path_to_yaml = '/work/westgroup/nora/Code/Advanced_Kinetics_Class/HW3/HW3_Q1.yaml'
gas = ct.Solution(path_to_yaml)

#T=list(np.linspace(298, 2000, 10000)) tried this range first to gauge
T=list(np.linspace(1311, 1312, 1000))

for temp in T:
    gas.TPX = temp, ct.one_atm, 'hexen_6_yl: .5, cyclohexyl: .5'
    if abs(gas.delta_standard_gibbs[0])<1: 
        print(temp, gas.delta_standard_gibbs[0]) #[J/kmol], this gets "Change in standard-state Gibbs free energy (independent of composition) for each reaction [J/kmol]"
       


1311.5095095095096 0.607162356376648


**The temperature at which Keq = 1 is ~1311.509 K.** 

# Question 2 

As mentioned by Prof. Green in class, we will assume that the TS of the reverse reaction has a similar geometry to cyclohexyl radical. 
We will be estimating the ring opening rate coefficient (cyclohexyl -> hexen_6_yl): 

From conventional TST:

### k = K_tunneling sym_num kBT/h exp(-∆Eo /kBT) qTS / Π qReactant,i

This is a unimolecular reaction, so equation simplifies to: 

### k = K_tunneling sym_num kBT/h exp(-∆Eo/kBT) qTS / qReactant

Here, Reactant = cyclohexyl radical.

To estimate the A factor:

* let's assume qTS/qReactant will be ~ 1 (likely a little higher, since the TS partition function is expected to be larger). We are assuming that the TS will look more similar to the cyclohexyl radical than the hexen_6_yl radical. 
* Let's assume tunneling doesn't matter since were at 1000K. 
* Assume symmetry number (really reaction path degeneracy) ~ 5 . 


Therefore, The A factor is ballparked to be = sym_num * kBT/h (really slightly higher). 

In [15]:
kb_div_h = 2.083661912*10**10 # Units of Hz/K
T = 1000
sym_num = 5 

A_factor_estimate = kb_div_h * 1000 * sym_num  
print('My A factor estimate is {:.4e} s^-1'.format(A_factor_estimate))

My A factor estimate is 1.0418e+14 s^-1


We know that the energy of the TS must be higher than any of the radicals. So let's assume that Ea >= (39.02700765 - 13.8969885 kcal/mol). The true Ea will likely be higher, since this is an estimate of the lower limit of Ea. 

In [32]:
delta_Eo = 39.02700765 - 13.8969885 #kcal/mol
R = 1.9872036e-3 #kcal.K-1.mol-1
T = 1000

estimate_of_exponential_term = np.exp(-delta_Eo/(R*T))

print('My estimate of the Ea is {:.4e} cal/mol'.format(-delta_Eo*1000))

My estimate of the Ea is -2.5130e+04 cal/mol


In [34]:
#now put it together to estimate the high-P limit rate coefficient
high_P_rate_coeff = A_factor_estimate * estimate_of_exponential_term
print('{:.4e}'.format(high_P_rate_coeff))

3.3554e+08


**The high-pressure-limit ring-opening rate coefficient for the cyclohexyl to
hexen-6-yl radical reaction at 1000 K is 3.3554e+08 s^-1**
 

In [26]:
#compare to JetSurf A factor (include in temp dependence before the exponential) 
T = 1000
JetSurf_A = 6.03e12 * (T**0.07) 
print('JetSurf\'s A factor estimate is {:.4e} s^-1. This differs by two orders of magnitude from my estimate.'.format(JetSurf_A))

JetSurf's A factor estimate is 9.7795e+12 s^-1. This differs by two orders of magnitude from my estimate.


JetSurf's Ea is 27983 cal/mol, which is very close to my Ea estimate of 25130.019 cal/mol.

In [ ]:
Ea = 27983/1000 #kcal/mol

JetSurf_exponential_term = np.exp(-Ea/(R*T)) 
JetSurf_K = JetSurf_A * JetSurf_exponential_term
print('The k for this reaction at 1000 K from the JetSurf mechanism is {:.4e} s^-1. This differs by two magnitudes from my estimate (3.3554e+08 s^-1)'.format(JetSurf_K))

**The k for this reaction at 1000 K from the JetSurf mechanism is 7.4948e+06 s^-1. This differs by two magnitudes from my estimate (3.3554e+08 s^-1).**

**This estimate does obey the bounds I set. JetSurf's Ea is greater than my estimated Ea (which was a lower bound), and the A factor is somewhat close to my estimate.**


# Question 3

* 1)     cyclohexyl radical -> hexen_6_yl
* 2)     hexen_6_yl -> cyclohexyl radical
* 3)     hexen_6_yl + NO -> CH2=CHCH2CH2CH2CH2NO

R1 = k1*[cyclohexyl radical]

R2 = k2*[hexen_6_yl]

R3 = k3*[hexen_6_yl]*[NO]


In [11]:
# step 1) Find thermo data for the acyclic nitroso product species.

#now let's calculate the thermo parameters using group additivity
acyclic_nitroso_product_species = Species(index=0, label='NO product', molecule=[Molecule(smiles='C=CCCCCN=O')])
acyclic_nitroso_product_parameters = database.thermo.get_thermo_data_from_groups(acyclic_nitroso_product_species)
acyclic_nitroso_product_parameters.to_nasa(298, 5000, 1000)

NASA(polynomials=[NASAPolynomial(coeffs=[2.00092,0.0662461,-4.07268e-05,1.25436e-08,-1.54144e-12,8627.5,24.589], Tmin=(298,'K'), Tmax=(1907.71,'K')), NASAPolynomial(coeffs=[21.1433,0.0261096,-9.16856e-06,1.51538e-09,-9.62425e-14,1323.79,-80.1264], Tmin=(1907.71,'K'), Tmax=(5000,'K'))], Tmin=(298,'K'), Tmax=(5000,'K'), E0=(70.6563,'kJ/mol'), Cp0=(33.2579,'J/(mol*K)'), CpInf=(436.51,'J/(mol*K)'), comment="""Thermo group additivity estimation: missing(O2d-N3d) + group(N3dOd-Cs) + group(Cs-CsCsHH) + group(Cs-CsCsHH) + group(Cs-(Cds-Cds)CsHH) + group(Cs-N3dCsHH) + group(Cds-CdsCsH) + group(Cds-CdsHH)""")

In [30]:
# get NO data 
NO_species = Species(index=0, label='NO', molecule=[Molecule(smiles='[N]=O')])
NO_parameters = database.thermo.get_thermo_data_from_library(NO_species, database.thermo.libraries['primaryThermoLibrary'])
NO_parameters[0].to_nasa(298, 5000, 1000)


NASA(polynomials=[NASAPolynomial(coeffs=[3.46128,-0.00040274,2.1655e-06,-1.54106e-09,3.42284e-13,9965.53,4.98532], Tmin=(298,'K'), Tmax=(1758.58,'K')), NASAPolynomial(coeffs=[4.19717,0.000330327,-5.12744e-07,2.52516e-10,-2.33289e-14,9334.53,-0.0384739], Tmin=(1758.58,'K'), Tmax=(5000,'K'))], Tmin=(298,'K'), Tmax=(5000,'K'), E0=(82.7753,'kJ/mol'), Cp0=(29.1007,'J/(mol*K)'), CpInf=(37.4151,'J/(mol*K)'), label="""NOJ""")

In [139]:
#now use Cantera to iterate through different initial mole fractions 
print('Partial Pressure of NO','\t', 'Percent of Hexen-6-yl Trapped', '\t', 'Mole Fractions of Initial Species')
path_to_yaml = '/work/westgroup/nora/Code/Advanced_Kinetics_Class/HW3/HW3_Q3.yaml'
gas = ct.Solution(path_to_yaml)

count=list(np.linspace(0, 1, 200)) 
mole_fraction_dicts=[f'hexen_6_yl: {x}, NO: {1-x}' for x in count]
#mole_fraction_dicts=['hexen_6_yl: .6, NO: .4']#, 'hexen_6_yl: .5, NO: .5', 'hexen_6_yl: .7, NO: .3' ]

for mole_fraction_dict in mole_fraction_dicts: 
    gas.TPX = 500, ct.one_atm, mole_fraction_dict
    r = ct.ConstPressureReactor(contents=gas, energy='off') #isothermal
    sim = ct.ReactorNet([r]) 
    #sim.verbose = True

    dt_max = 1.e-7 
    t_end = 1 * dt_max
    states = ct.SolutionArray(gas, extra='t')


    while sim.time < t_end:
        sim.step()
        states.append(r.thermo.state, t=sim.time)

    initial_hex_rad_conc = states.concentrations[:,gas.species_index('hexen_6_yl')][0]
    ending_NOproduct_conc = states.concentrations[:,gas.species_index('acyclic_nitroso_product')][-1]
    
    if (ending_NOproduct_conc/initial_hex_rad_conc < .06) and (ending_NOproduct_conc/initial_hex_rad_conc)>.04:
        print((states.X[:,gas.species_index('NO')][0])*gas.P/101325, '\t', '{:5f} %'.format(ending_NOproduct_conc/initial_hex_rad_conc), '\t\t', mole_fraction_dict)
 


Partial Pressure of NO 	 Percent of Hexen-6-yl Trapped 	 Mole Fractions of Initial Species


/work/westgroup/nora/miniconda/envs/rmg_env/lib/python3.7/site-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in double_scalars


0.050251256268534356 	 0.055710 % 		 hexen_6_yl: 0.949748743718593, NO: 0.05025125628140703
0.045226130640936744 	 0.049612 % 		 hexen_6_yl: 0.9547738693467337, NO: 0.045226130653266305
0.040201005013389905 	 0.043639 % 		 hexen_6_yl: 0.9597989949748744, NO: 0.04020100502512558


## **The Cantera simulation shows that when the initial partial pressure of NO is 0.38 atm, 1% of the ring-opened species hexen-6-yl is trapped**

In [ ]:
#now use Cantera to iterate through different initial mole fractions 
path_to_yaml = '/work/westgroup/nora/Code/Advanced_Kinetics_Class/HW3/HW3_Q3.yaml'
gas = ct.Solution(path_to_yaml)

#count=list(np.linspace(0, 1, 100)) 
#mole_fraction_dicts=[f'hexen_6_yl: {x}, NO: {1-x}' for x in count]
mole_fraction_dicts=['hexen_6_yl: .7, NO: .3']#, 'hexen_6_yl: .5, NO: .5', 'hexen_6_yl: .7, NO: .3' ]

for mole_fraction_dict in mole_fraction_dicts: 
    gas.TPX = 298, ct.one_atm, mole_fraction_dict
    r = ct.ConstPressureReactor(contents=gas)#, energy='off') 
    sim = ct.ReactorNet([r]) 
    sim.verbose = True

    dt_max = 1.e-5 
    t_end = 1 * dt_max
    states = ct.SolutionArray(gas, extra='t')


    while gas.T <= 500:
        sim.step()
        states.append(r.thermo.state, t=sim.time)

    initial_hex_rad_conc = states.concentrations[:,gas.species_index('hexen_6_yl')][0]
    ending_NOproduct_conc = states.concentrations[:,gas.species_index('acyclic_nitroso_product')][-1]

    print(states.concentrations[:,gas.species_index('hexen_6_yl')][0], states.concentrations[:,gas.species_index('acyclic_nitroso_product')][-1])    
    #if 
# df = pd.DataFrame(zip(states.X[:,gas.species_index('C2H6(36)')], states.t))


Initializing reactor network.
Reactor 0: 6 variables.
              0 sensitivity params.
Number of equations: 6
Maximum time step:                0
0.02862623308939047 0.004550314521288396


# Question 4

In [145]:
# let's make an inert environment with Argon

# get Ar data 
Ar_species = Species(index=0, label='Ar', molecule=[Molecule(smiles='[Ar]')])
Ar_parameters = database.thermo.get_thermo_data_from_library(Ar_species, database.thermo.libraries['primaryThermoLibrary'])
Ar_parameters[0]


NASA(polynomials=[NASAPolynomial(coeffs=[2.5,0,0,0,0,-745.375,4.37967], Tmin=(200,'K'), Tmax=(1000,'K')), NASAPolynomial(coeffs=[2.5,0,0,0,0,-745.375,4.37967], Tmin=(1000,'K'), Tmax=(6000,'K'))], Tmin=(200,'K'), Tmax=(6000,'K'), Cp0=(20.7862,'J/(mol*K)'), CpInf=(20.7862,'J/(mol*K)'), label="""Ar""")

In [203]:
#now use Cantera to iterate through different initial mole fractions 
print('Mole Fractions\t\t\t\t\t\tRing Opening Rate','\t', 'NO attachment Rate', '\tPartial Pressure of NO')
path_to_yaml = '/work/westgroup/nora/Code/Advanced_Kinetics_Class/HW3/HW3_Q4.yaml'
gas = ct.Solution(path_to_yaml)

count=list(np.linspace(0, 1, 200)) 
mole_fraction_dicts=[f'hexen_6_yl: .1, NO: {(1-x)/10}, Argon: .8' for x in count]
#mole_fraction_dicts=['hexen_6_yl: .6, NO: .4']#, 'hexen_6_yl: .5, NO: .5', 'hexen_6_yl: .7, NO: .3' ]

for mole_fraction_dict in mole_fraction_dicts: 
    gas.TPX = 1000, ct.one_atm, mole_fraction_dict
    r = ct.ConstPressureReactor(contents=gas, energy='off') #isothermal
    sim = ct.ReactorNet([r]) 
    #sim.verbose = True

    dt_max = 1.e-7 
    t_end = 1 * dt_max
    states = ct.SolutionArray(gas, extra='t')


    while sim.time < t_end:
        sim.step()
        states.append(r.thermo.state, t=sim.time)

    reac_1_rate =  gas.reverse_rates_of_progress[0] #hex -> cyclo
    reac_2_rate =  gas.forward_rates_of_progress[1] #NO attachment
      
    if reac_1_rate > reac_2_rate: 
        print(mole_fraction_dict,'\t', gas.reverse_rates_of_progress[0], '\t',gas.forward_rates_of_progress[1], '\t', (states.X[:,gas.species_index('NO')][0]*gas.P)/101325)


Mole Fractions						Ring Opening Rate 	 NO attachment Rate 	Partial Pressure of NO
hexen_6_yl: .1, NO: 0.003015075376884424, Argon: .8 	 263.0378042422054 	 257.3341751421223 	 0.0033388981634357
hexen_6_yl: .1, NO: 0.0025125628140703514, Argon: .8 	 263.8157709185349 	 214.17644312730425 	 0.0027839643651139854
hexen_6_yl: .1, NO: 0.002010050251256279, Argon: .8 	 266.92760879514003 	 169.51038130716614 	 0.002228412256153528
hexen_6_yl: .1, NO: 0.0015075376884422064, Argon: .8 	 268.19710655381914 	 126.71489887615176 	 0.0016722408025900434
hexen_6_yl: .1, NO: 0.0010050251256281339, Argon: .8 	 264.2127414640298 	 86.01903274694283 	 0.0011154489681525878
hexen_6_yl: .1, NO: 0.0005025125628140726, Argon: .8 	 267.5609337904827 	 42.51015242892498 	 0.0005580357142571199
hexen_6_yl: .1, NO: 0.0, Argon: .8 	 268.70989304360177 	 0.0 	 0.0


**If you wanted to measure the equilibrium, rather than the ring-closing rate coefficient, at 1000 K, we would use partial pressure of NO that equals 0.0033 atms or less. At this partial pressure, the NO attachment rate becomes slower than the ring-opening reaction rate (as shown above).** 

# Question 5 

In [ ]:
Note that in addition to the reactions mentioned above, all the radicals can recombine
and they all can react with NO, all with k’s around 1e13 cm3
/mole-s. Also there are two
different ring-closed radicals that can be formed by the hexenyl radical (the other one has
a 5-membered ring, we expect it formed with a similar rate coefficent as the 6-membered
ring). Do these possible side reactions cause any complications in your proposed
experiments from parts 3 and 4? Briefly explain. How could you check numerically
to see if these side reactions significantly mess up your ability to determine the k and
Keq using these measurements?

**Do these possible side reactions cause any complications in your proposed experiments from parts 3 and 4? Briefly explain.**


Yes, these side reactions DO cause complications in the proposed experiment for 3 and 4 for the following reasons. 

* we assume that the only species reacting with NO is the hexen-6-yl radical. Depending on the amount of NO-involving side reactions and the initial amount of NO, we could run into a problem where NO is not in abundance (since it is being used up in BOTH the hexen-6-yl + NO reaction and all the side reactions), thus we could get an artificially deflated amount of CH2=CHCH2CH2CH2CH2NO since NO is limited. 

* these other side reactions (specifically radical recombination followed by reaction with NO) could possibly generate CH2=CHCH2CH2CH2CH2NO, which would artificially inflate the amount of this species. 

These two bullets are problematic since our experiment relies on measuring CH2=CHCH2CH2CH2CH2NO that is solely generated from the hexen-6-yl + NO reaction and assumes there is enough NO in the system for this reaction to proceed for each for hexen-6-yl radical. 

**How could you check numerically to see if these side reactions significantly mess up your ability to determine the k and Keq using these measurements?**

To check numerically, you could add "example" side reactions reactions with plausible kinetics to the system and see if that changes your initially calculated k and Keqs. 
